In [1]:
!pip3 install torch_optimizer torchmetrics
!nvidia-smi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 636.5 kB/s eta 0:00:00a 0:00:01
Wed Sep 21 00:24:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_optimizer import Ranger
from torchvision import datasets
from torchvision import transforms,models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchmetrics import Accuracy
from torch.optim.lr_scheduler import CyclicLR

torch.manual_seed(43)

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:

def target_trans(target):
  y = -torch.ones(100)
  y[target] = 1

  return y,target
  
'''
  global labels
  label = labels[target]
  for j in range(27):
    if label in get_descendants(labels[100+j]):
      y[100+j] = 1
'''
  

'\n  global labels\n  label = labels[target]\n  for j in range(27):\n    if label in get_descendants(labels[100+j]):\n      y[100+j] = 1\n'

In [5]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=[32,32], padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.CenterCrop(size=[32,32]),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = datasets.CIFAR100(root='data/', download=True, transform=transform_train,target_transform=target_trans)
test_dataset = datasets.CIFAR100(root='data/', train=False, transform=transform_test,target_transform=target_trans)


val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
batch_size=256

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(val_ds, batch_size, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size, num_workers=4)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data/


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [6]:
class Hinge_Loss(torch.nn.Module):
    
    def __init__(self):
        super(Hinge_Loss,self).__init__()
        
    def forward(self,y_p,y_t):

        temp = 1 - y_p * y_t
        
        zero = torch.zeros(y_p.size()[0],y_p.size()[1])
        zero = zero.cuda()
        clamp = torch.max(temp,zero)
    
        total_loss = torch.sum(clamp)/y_p.size()[0]
        return total_loss

In [7]:
num_classes = 100
num_epochs = 100


In [8]:
class VGGForCiFar100(nn.Module):
  def __init__(self, hid_dim=5000, dropout=0.5, n_classes=100, use_fc=True, freeze=False):
    super().__init__()
    self.vgg = models.vgg16(pretrained=True)
    if not use_fc:
      self.vgg.classifier = nn.Linear(512 * 7 * 7, n_classes)
    else:
      self.vgg.classifier = nn.Sequential(
              nn.Linear(512 * 7 * 7, 4096),
              nn.ReLU(True),
              nn.Dropout(p=dropout),
              nn.Linear(4096, 4096),
              nn.ReLU(True),
              nn.Dropout(p=dropout),
              nn.Linear(4096, n_classes),
          )
      
    if freeze:
      for param in self.vgg.features.parameters():
        param.requires_grad = False
      for param in self.vgg.avg_pool.parameters():
        param.requires_grad = False
      for param in self.vgg.flatten.parameters():
        param.requires_grad = False
    

  def forward(self, x):
    return F.log_softmax(self.vgg(x), dim=-1)

In [9]:

model = VGGForCiFar100(n_classes=num_classes).to(device)
criterion = Hinge_Loss().to(device)
accuracy = Accuracy(num_classes=num_classes).to(device)
optimizer = Ranger(model.parameters(), lr=1e-3, weight_decay=1e-4) 
scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-3, step_size_up=len(train_loader)//2, cycle_momentum=False)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [10]:
model

VGGForCiFar100(
  (vgg): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [11]:

def get_accuracy(output, target):

  batch_size = target.size(0)
  correct=0

  pred = output.max(dim=1)[1]

  correct = pred==target

  acc = correct.float().sum(0)

  return acc/batch_size


In [12]:
total_train_step = len(train_loader)
#print(total_train_step)
total_val_step=len(valid_loader)
BEST_VAL_METRIC = 0
BEST_MODEL = None

for epoch in range(1, num_epochs+1):

    train_loss=0
    train_acc=0.0
    model.train()

    for i, (images, target) in enumerate(train_loader, 1):

        y_trans = target[0]
        y_true = target[1]

        # Move tensors to the configured device
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, y_trans)

        train_loss += loss
        #train_acc += get_accuracy(outputs, y_true)
        train_acc += accuracy(outputs, y_true)
        
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    print(f'Epoch [{epoch}/{num_epochs}] - Loss: {(train_loss/total_train_step):.4f}, Accuracy: {(train_acc/total_train_step):.4f}')

    model.eval() 
    # Validation
    with torch.no_grad():
        val_acc = 0
        val_loss=0
        for i, (images, target) in enumerate(valid_loader, 1):

            y_trans = target[0]
            y_true = target[1]

            # Move tensors to the configured device
            images = images.to(device)
            y_true = y_true.to(device)
            y_trans = y_trans.to(device)

            outputs = model(images)
            val_loss += criterion(outputs, y_trans)
            #val_acc += get_accuracy(outputs, y_true)
            val_acc += accuracy(outputs, y_true)

    if val_acc/total_val_step > BEST_VAL_METRIC:
        BEST_VAL_METRIC = val_acc/total_val_step
        BEST_MODEL = model.state_dict() 

    print(f'Accuracy of the network on the 5000 validation images: {(val_acc/total_val_step):.4f}, loss: {(val_loss/total_val_step):.4f}') 

/opt/conda/lib/python3.7/site-packages/pytorch_ranger/ranger.py:172: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /usr/local/src/pytorch/torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


Epoch [1/100] - Loss: 4.0942, Accuracy: 0.2603
Accuracy of the network on the 5000 validation images: 0.4612, loss: 3.0160
Epoch [2/100] - Loss: 3.0833, Accuracy: 0.4529
Accuracy of the network on the 5000 validation images: 0.5186, loss: 2.7556
Epoch [3/100] - Loss: 2.7946, Accuracy: 0.5261
Accuracy of the network on the 5000 validation images: 0.5631, loss: 2.6524
Epoch [4/100] - Loss: 2.6180, Accuracy: 0.5733
Accuracy of the network on the 5000 validation images: 0.5817, loss: 2.5943
Epoch [5/100] - Loss: 2.5020, Accuracy: 0.6024
Accuracy of the network on the 5000 validation images: 0.5957, loss: 2.5241
Epoch [6/100] - Loss: 2.4065, Accuracy: 0.6272
Accuracy of the network on the 5000 validation images: 0.6046, loss: 2.5068
Epoch [7/100] - Loss: 2.3066, Accuracy: 0.6534
Accuracy of the network on the 5000 validation images: 0.6176, loss: 2.4844
Epoch [8/100] - Loss: 2.2184, Accuracy: 0.6758
Accuracy of the network on the 5000 validation images: 0.6206, loss: 2.4596
Epoch [9/100] - 

In [13]:
#Testing
model.load_state_dict(BEST_MODEL)

total_test_step=len(test_loader)

with torch.no_grad():
    test_acc=0
    test_loss=0

    for i, (images, target) in enumerate(test_loader, 1):
        
        y_trans = target[0]
        y_true = target[1]
        
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)

        # Forward pass
        outputs = model(images)
        
        # Loss
        test_loss += criterion(outputs,y_trans)
        test_acc += accuracy(outputs, y_true)

    print(f'Accuracy of the network on test images: {(test_acc/total_test_step):.4f}, loss: {(test_loss/total_test_step):.4f}')

Accuracy of the network on test images: 0.6615, loss: 2.9168
